In [1]:
import os
import shutil
import glob
import pandas as pd
import geopandas as gpd
import dask
from dask.distributed import Client, progress, LocalCluster
from pyFIRS.wrappers import lastools, fusion
from pyFIRS.utils import clean_dir, clean_buffer_polys, clip_tile_from_shp, convert_project

/storage/anaconda3/envs/pyFIRS/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


### Setting up parallel computing using `dask.distributed`
`LAStools` offers native multi-core processing as an optional argument (`cores`) supplied to its command-line tools. `FUSION` command line tools do not. To enable parallel processing of `FUSION` commands, we'll use `dask.distributed` to schedule the processing of tiles in asynchronous parallel batches. This approach also offers us the ability to track progress using a progress bar.

You'll first need to launch a parallel computing cluster. 

In [2]:
cluster=LocalCluster(scheduler_port=7001, diagnostics_port=7002)
c = Client(cluster)

At this point, you should also be able to view an interactive dashboard on port 7002. If you're executing this on a remote server, you'll need to set up port forward so you can view the dashboard on your local machine's browser. Once you've done that, or if you're processing on your own machine, you can view the dashboard at [http://localhost:7002/status](http://localhost:7002/status).

# Enough already, let's get to work with some lidar data

We'll define where we can find the binary executables for LAStools and FUSION command line tools.

In [4]:
las = lastools.useLAStools('/storage/lidar/LAStools/bin')
fus = fusion.useFUSION('/storage/lidar/FUSION/')

We'll create working directories for raw (imported with modest clean-up from source files), interim, and processed data.

In [5]:
# where the imported lidar data is currently stored
workdir = os.path.abspath('/storage/lidar/olc_metro_2014')

# the coordinate reference system we'll be working with
target_epsg = 26910 # utm 10 N

In [6]:
# define data handling directories
raw, interim, processed = os.path.join(workdir,'raw'), os.path.join(workdir,'interim'), os.path.join(workdir,'processed')
layers = os.path.join(interim, 'layers')

num_cores = len(c.ncores()) # identify how many workers we have

# push our working directories and wrapper classes to the workers on the cluster as well
c.scatter([src_dir, raw, interim, processed, layers, las, fus, target_epsg, num_cores], broadcast=True);

## 0. Retile the data to add buffers for avoiding edge effects during processing.

In practice, executing the `lastile` command on individual tiles in parallel is likely to corrupt your output files. I suspect this is because the dynamic re-tiling of input files means that many output tiles are likely to require inputs from multiple input files, and that parallel processing outside of LAStools may result in collisions writing data from multiple inputs to these output tiles. So, for this case, we'll let `lastile` handle the parallelism under the hood. We won't have a progress bar, but this shouldn't take more than 5-10 minutes per ~100 tiles (with vendor tile size ~1000x1000m with 4-8 pts/m2).

**THERE ARE ARGUMENTS IN THE FOLLOWING COMMAND THAT DEPEND UPON THE UNITS OF THE DATA.**

In [ ]:
%%time
tile_proc = las.lastile(i=os.path.join(raw, '*.laz'),
                        tile_size=1000, # in units of lidar data
                        buffer=25, # assumes units are in meters
                        flag_as_withheld=True, # flag buffer points as "withheld", enables handling with other LAStools
                        extra_pass=True, # if outputting to LAZ format, can help avoid memory limits
                        full_bb=True,
                        olas=True,
                        odir=os.path.join(interim, 'retiled'),
                        cores=num_cores);

If you want to confirm that the point cloud files you've retiled haven't been corrupted (i.e., they still match valid LAS format specifications), you can use the following function.

In [ ]:
@dask.delayed
def validate(infile): # the function we'll map to a list of inputs
    odir, basename = os.path.split(infile)
    tile_id = basename.split('.')[0]
    outfile = os.path.join(odir, tile_id + ".xml")
    if os.path.exists(outfile):
        pass
    else:
        proc = las.lasvalidate(i=infile,
                               o=outfile)
    return tile_id

infiles = glob.glob(os.path.join(interim, 'retiled', '*.las'))
val_results = c.persist([validate(file) for file in infiles])
progress(val_results)

In [ ]:
# check to see if any of the files failed validation
# if a file shows up here, inspect the .xml file and see whether the corruption is a concern
# for example, in this acquisition, the corruption issue is GPS time, which we aren't worried about
corrupted = [val_results[i].result().args[3] for i in range(len(val_results)) if 'fail' in val_results[i].result().stderr.decode()]
corrupted

## 1. Classify points in the lidar point cloud

If the original tiles delivered by the vendor included overlapping edges, our retiling may result in duplicated points in the new tiles from overlapping edges of vendor-provided input tiles. In the next step, we will ensure that only one point with unique (X,Y,Z) coordinates are retained in the point cloud data.

In [ ]:
@dask.delayed
def remove_dupes(tile_id):
    proc = las.lasduplicate(i=os.path.join(interim, 'retiled', tile_id + '.las'),
                            unique_xyz=True,
                            olay=True,
                            olaydir=layers,
                            odir=layers)
    return tile_id

Now we'll remove points that are isolated as likely noise.

In [ ]:
@dask.delayed
def denoise(tile_id):    
    proc = las.lasnoise(i=os.path.join(interim, 'retiled', tile_id + '.las'),
                        remove_noise=True,
                        ilaydir=layers,
                        olay=True,
                        olaydir=layers)
    return tile_id

Next, calculate the height aboveground for each point for use in classifying them.

In [ ]:
@dask.delayed
def hag(tile_id):
    proc = las.lasheight(i=os.path.join(interim, 'retiled', tile_id + '.las'),
                         ilaydir=layers,
                         olay=True,
                         olaydir=layers)
    return tile_id

Now, we'll classify points as building or high vegetation that meet certain criteria for 'planarity' or 'ruggedness'. 

**THERE ARE ARGUMENTS IN THE FOLLOWING COMMAND THAT DEPEND UPON THE UNITS OF THE DATA.**

If your data are in meters, you should change these parameters, or consider reprojecting the data to a projection that is in feet when you copy the source data into our working directory using the `import_tiles` / `las2las` command at the top of this notebook.

In [ ]:
@dask.delayed
def classify(tile_id):
    proc = las.lasclassify(i=os.path.join(interim, 'retiled', tile_id + '.las'),
                           ilaydir=layers,
                           step=2.0, # if your data are in meters
                           planar=0.1, # if your data are in meters
                           rugged=0.4, # if your data are in meters
                           olas=True,
                           odir=os.path.join(interim, 'classified'))
    return tile_id

We'll now remove the points in the buffered area of each tile and put the clean tiles in the processed folder.

In [ ]:
@dask.delayed
def dropwithheld(tile_id):
    outfile = os.path.join(processed, 'points', tile_id + '.las')
    if not os.path.exists(outfile):
        proc = las.las2las(i=os.path.join(interim, 'classified', tile_id + '.las'),
                           drop_withheld=True, # remove points flagged as withhled
                           set_user_data=0, # remove height aboveground calculated using lasheight
                           olaz=True, # compress outputs to LAZ format
                           odir=os.path.join(processed, 'points'))
    else:
        pass
    
    return tile_id

We'll produce a shapefile showing the layout of the tiles as a single shapefile. This is a single process that takes a few seconds to run, so no need to distribute it using `dask`.

In [ ]:
@dask.delayed
def tiles_overview(*args, **kwargs):
    odir = os.path.join(processed, 'vectors')
    
    if os.path.exists(os.path.join(processed, 'vectors', 'tiles.shp')):
        pass
    else:
        proc = las.lasboundary(i=os.path.join(processed, 'points', '*.laz'),
                               use_bb=True, # use bounding box of tiles
                               overview=True,
                               labels=True,
                               cores=num_cores, # use parallel processing
                               oshp=True,
                               o=os.path.join(processed, 'vectors', 'tiles.shp'))
    return

## 2. Generate a bare earth Digital Elevation Model
Generate tiles of the bare earth model. This assumes that there are already ground-classified points

In [ ]:
@dask.delayed
def make_dem(tile_id):
    proc = las.las2dem(i=os.path.join(interim, 'classified', tile_id + '.las'),
                       odir=os.path.join(processed, 'rasters', 'DEM_tiles'),
                       otif=True, # create tiles as GeoTiff rasters
                       keep_class=2, # keep ground-classified returns only
                       step=1, # resolution of output raster, in units of lidar data
                       thin_with_grid=1, # use a 1 x 1 resolution for creating the TIN for the DEM
                       extra_pass=True, # uses two passes over data to execute DEM creation more efficiently
                       use_tile_bb=True) # remove buffers from tiles
    return tile_id

In [ ]:
@dask.delayed
def add_crs_dem(tile_id):
    basename = tile_id + '.tif'
    infile = os.path.join(processed, 'rasters', 'DEM_tiles', basename)
    
    proc = subprocess.run(['rio', 'edit-info', '--crs', 'EPSG:{}'.format(target_epsg), infile],
                   stderr=subprocess.PIPE, stdout=subprocess.PIPE)
    
    return tile_id

To create a hillshade layer, we'll first, generate hillshade tiles from the bare earth model.

In [ ]:
@dask.delayed
def make_hillshade(tile_id):
    proc = las.las2dem(i=os.path.join(interim, 'classified', tile_id + '.las'),
                       odir=os.path.join(processed, 'rasters', 'hillshade_tiles'),
                       otif=True, # create tiles as GeoTiffs
                       hillshade=True,
                       keep_class=2, # keep ground-classified returns only
                       step=1, # resolution of output raster, in units of lidar data
                       thin_with_grid=1, # use a 0.5 x 0.5 resolution for creating the TIN for the DEM
                       extra_pass=True, # uses two passes over data to execute DEM creation more efficiently
                       use_tile_bb=True) # remove buffers from tiles
    return tile_id

In [ ]:
@dask.delayed
def add_crs_hill(tile_id):
    basename = tile_id + '.tif'
    infile = os.path.join(processed, 'rasters', 'hillshade_tiles', basename)
    
    proc = subprocess.run(['rio', 'edit-info', '--crs', 'EPSG:{}'.format(target_epsg), infile],
                          stderr=subprocess.PIPE, stdout=subprocess.PIPE)
    
    return tile_id

## 3. Identify building footprints
First start by building shapefiles showing building boundaries in each buffered tile.

In [ ]:
@dask.delayed
def bldg_tiles(tile_id):
    proc = las.lasboundary(i=os.path.join(interim, "retiled", tile_id + '.las'),
                           ilaydir=layers,
                           keep_class=6, # use only building-classified points
                           disjoint=True, # compute separate polygons for each building
                           concavity=1, # map concave boundary if edge length >= 1m
                           oshp=True,
                           odir=os.path.join(interim, 'building_tiles'))
    return tile_id

Generate shapefiles showing the bounding box of each (unbuffered) tile that we'll use to remove buildings that fall in the buffered area.

In [ ]:
@dask.delayed
def bbox_shp(tile_id):
    basename = tile_id + '.laz'
    infile = os.path.join(processed, 'points', basename)
    odir = os.path.join(interim, 'tile_boundaries')
    
    if os.path.exists(os.path.join(odir, basename)):
        pass
    else:
        proc = las.lasboundary(i=infile,
                               odir=odir,
                               oshp=True,
                               use_tile_bb=True)
    return tile_id

For each shapefile containing polygons of the building boundaries, we'll use the `clean_buffer_polys` function from `pyFIRS.utils` to remove polygons from a tile if their centroid falls in the buffered area of the tile.

In [ ]:
@dask.delayed
def clean_bldgs(tile_id, *args):
    if type(tile_id) == list:
        tile_id = tile_id[0]
    basename = tile_id + '.shp'
    infile = os.path.join(interim, 'building_tiles', basename)
    tile_shp = os.path.join(interim, 'tile_boundaries', basename)
    odir = os.path.join(processed, 'vectors', 'building_tiles')
    
    if os.path.exists(os.path.join(odir, basename)):
        pass
    else:
        clean_buffer_polys(infile,
                           tile_shp,
                           odir=odir,
                           simp_tol=3,
                           simp_topol=True)
    return tile_id

## 4. Create a Canopy Height Model
We're going to switch use a FUSION command line tool to generate a Canopy Height Models (CHMs). 

### Using FUSION's `canopymodel` to generate CHMs
`FUSION` wants to have ground models formatted as .dtm files, for CHM development and for estimating other canopy metrics. Let's generate these ground models first using a 1-meter x-y resolution.

In [ ]:
@dask.delayed
def groundDTMs(tile_id):
    infile = os.path.join(interim, 'classified', tile_id + '.las')
    odir = os.path.join(interim, 'dtm_ground_tiles')
    outfile = os.path.join(odir, tile_id + '.dtm')
    
    if os.path.exists(os.path.join(odir, tile_id + '.laz')):
        pass
    else:
        proc = fus.gridsurfacecreate(surfacefile=outfile,
                                     cellsize=1,
                                     xyunits='M',
                                     zunits='M',
                                     coordsys=1, # in UTM
                                     zone='10N',
                                     horizdatum=2, # NAD83
                                     vertdatum=2, # NAVD88
                                     datafile=infile,
                                     las_class=2, # keep only ground-classified points
                                     odir=odir) # will make sure output directory is created if doesn't already exist
    return tile_id

In [ ]:
@dask.delayed
def canopymodel(tile_id):
    infile = os.path.join(interim, 'classified', tile_id + '.las')
    odir = os.path.join(interim, 'chm_tiles')
    outfile = os.path.join(odir, tile_id + '.dtm')
    
    if os.path.exists(os.path.join(odir, tile_id + '.dtm')):
        pass
    else:
        proc = fus.canopymodel(surfacefile=outfile,
                               cellsize=0.5, # in meters
                               xyunits='M',
                               zunits='M',
                               coordsys=1, # in UTM
                               zone='10N', # not in UTM
                               horizdatum=2, # NAD83
                               vertdatum=2, # NAVD88
                               datafiles=infile,
                               ground=os.path.join(interim, 'dtm_ground_tiles', tile_id + '.dtm'),
                               median=3, # median smoothing in 3x3 kernel
                               las_class=(1,2,5), # keep only ground, unclassified, and high veg points
                               asc=True, # also output in ascii format
                               odir=odir) # will make sure output directory is created if doesn't already exist
    return tile_id

Convert the ascii files that `canopymodel` generated into GeoTiffs, specifying their projection. Then cleanup the files `canopymodel` generated that we don't care about.

In [ ]:
@dask.delayed
def asc2tif(tile_id):
    infile = os.path.join(interim, 'chm_tiles', tile_id + '.asc')
    
    if os.path.exists(os.path.join(interim, 'chm_tiles', tile_id + '.tif')):
        pass
    else:
        convert_project(infile, '.tif', 'EPSG:{}'.format(target_epsg))
    
    return tile_id

Clip the canopy height model tiles to remove overlapping areas that were from tile buffering to avoid edge effects.

In [ ]:
@dask.delayed
def clip(tile_id, *args):
    if type(tile_id) == list:
        tile_id = tile_id[0]
        
    infile = os.path.join(interim, 'chm_tiles', tile_id + '.tif')
    in_shp = os.path.join(interim, 'tile_boundaries', tile_id + '.shp')
    odir = os.path.join(processed, 'rasters', 'chm_tiles')
    
    if os.path.exists(os.path.join(odir, tile_id + '.tif')):
        pass
    else:
        clip_tile_from_shp(infile, in_shp, odir)
    
    return tile_id

In [ ]:
@dask.delayed
def tile_done(tile_id, *args, **kwargs):
    return tile_id

@dask.delayed
def tiles_done(*args, **kwargs):
    return

## Hand-build the computational graph
Define the recipe for computations.

In [ ]:
tile_ids = [os.path.basename(file).split('.')[0] for file in glob.glob(os.path.join(interim, 'retiled', '*.las'))]
print('Found {:,d} tiles to process'.format(len(tile_ids)))
print(tile_ids[0:5])

The computational pipeline, including dependencies of each step in the pipeline, can be specified using a dictionary. 

In [ ]:
dsk = {}
for tile in tile_ids:
    dsk['dedupe-{}'.format(tile)]=(remove_dupes, tile)
    dsk['denoise-{}'.format(tile)]=(denoise, 'dedupe-{}'.format(tile))
    dsk['normalize-{}'.format(tile)]=(hag, 'denoise-{}'.format(tile))
    dsk['classify-{}'.format(tile)]=(classify, 'normalize-{}'.format(tile))
    dsk['drop-{}'.format(tile)]=(dropwithheld, 'classify-{}'.format(tile))
    dsk['bbox-{}'.format(tile)]=(bbox_shp, 'drop-{}'.format(tile))
    dsk['dem-{}'.format(tile)]=(make_dem, 'classify-{}'.format(tile))
    dsk['prj_dem-{}'.format(tile)]=(add_crs_dem, 'dem-{}'.format(tile))
    dsk['hill-{}'.format(tile)]=(make_hillshade, 'classify-{}'.format(tile))
    dsk['prj_hill-{}'.format(tile)]=(add_crs_hill, 'hill-{}'.format(tile))
    dsk['bldgs_buff-{}'.format(tile)]=(bldg_tiles, 'classify-{}'.format(tile))
    dsk['bldgs_clean-{}'.format(tile)]=(clean_bldgs, ['bldgs_buff-{}'.format(tile), 'bbox-{}'.format(tile)])
    dsk['ground_dtm-{}'.format(tile)]=(groundDTMs, 'classify-{}'.format(tile))
    dsk['canopy-{}'.format(tile)]=(canopymodel, 'ground_dtm-{}'.format(tile))
    dsk['canopy_tif-{}'.format(tile)] = (asc2tif, 'canopy-{}'.format(tile))
    dsk['canopy_clip-{}'.format(tile)]=(clip, ['canopy_tif-{}'.format(tile), 'bbox-{}'.format(tile)])
    dsk['tile_done-{}'.format(tile)]=(tile_done, ['denoise-{}'.format(tile), 
                                                  'normalize-{}'.format(tile), 
                                                  'classify-{}'.format(tile), 
                                                  'drop-{}'.format(tile),
                                                  'bbox-{}'.format(tile),
                                                  'dem-{}'.format(tile),
                                                  'prj_dem-{}'.format(tile),
                                                  'hill-{}'.format(tile),
                                                  'prj_hill-{}'.format(tile),
                                                  'bldgs_buff-{}'.format(tile),
                                                  'bldgs_clean-{}'.format(tile),
                                                  'ground_dtm-{}'.format(tile),
                                                  'canopy-{}'.format(tile),
                                                  'canopy_tif-{}'.format(tile),
                                                  'canopy_clip-{}'.format(tile)])
    
dsk['tiles_done'] = (tiles_done, ['tile_done-{}'.format(tile) for tile in tile_ids])

In [ ]:
processing_steps = ['dedupe', 'denoise', 'normalize', 'classify', 'drop', 'bbox', 
                    'dem', 'prj_dem', 'hill', 'prj_hill', 'bldgs_buff', 
                    'bldgs_clean', 'ground_dtm', 'canopy', 'canopy_tif', 'canopy_clip']

In [ ]:
example_tile_graph = c.get(dsk, 'tile_done-{}'.format(tile_ids[0]))
example_tile_graph.visualize()

In [ ]:
# example_tile_results = c.compute(example_tile_graph) # this might take a while...
# progress(example_tile_results)

Get Dask to determine how to get to the last state of the tile-processing pipeline, building a computational graph.

In [ ]:
tiles_graph = c.get(dsk, 'tiles_done')

In [ ]:
tiles_results = c.compute(tiles_graph) # this might take a while...
progress(tiles_results)

In [ ]:
%%time
# get rid of the .tfw and .kml files that LAStools generates
clean_dir(os.path.join(processed, 'rasters', 'DEM_tiles'), ['.tfw', '.kml'])

In [ ]:
%%time
# get rid of the .tfw and .kml files that LAStools generates
clean_dir(os.path.join(processed, 'rasters', 'hillshade_tiles'), ['.tfw', '.kml'])

In [ ]:
%%time
# get rid of the .asc and .dtm files that FUSION generates
clean_dir(os.path.join(interim, 'chm_tiles'), ['.asc', '.dtm'])

# Merge tiled derivative outputs together
Merge all the tiled GeoTiffs and Shapefiles into single overview files.

Merge the bare earth tiles into a single GeoTiff.

In [ ]:
@dask.delayed
def merge_dem(*args, **kwargs):
    infiles = os.path.join(processed, 'rasters', 'DEM_tiles', '*.tif')
    outfile = os.path.join(processed, 'rasters', 'dem.tif')
    
    if os.path.exists(outfile):
        return
    else:
        return subprocess.run(['rio', 'merge', *glob.glob(infiles), outfile, '--co', 'compress=LZW',
                              '--co', 'tiled=true', '--co', 'blockxsize=256', '--co', 'blockysize=256'],
                              stderr=subprocess.PIPE, stdout=subprocess.PIPE)

Now merge the hillshade tiles into a single raster formatted as GeoTiff.

In [ ]:
@dask.delayed
def merge_hillshade(*args, **kwargs):
    infiles = os.path.join(processed, 'rasters', 'hillshade_tiles', '*.tif')
    outfile = os.path.join(processed, 'rasters', 'hillshade.tif')

    if os.path.exists(outfile):
        return
    else:
        return subprocess.run(['rio', 'merge', *glob.glob(infiles), outfile, '--co', 'compress=LZW',
                              '--co', 'tiled=true', '--co', 'blockxsize=256', '--co', 'blockysize=256'],
                              stderr=subprocess.PIPE, stdout=subprocess.PIPE)

Merge the trimmed canopy height model tiles into a single raster.

In [ ]:
@dask.delayed
def merge_chm(*args, **kwargs):
    infiles = os.path.join(processed, 'rasters', 'chm_tiles', '*.tif')
    outfile = os.path.join(processed, 'rasters', 'chm.tif')
    
    if os.path.exists(outfile):
        pass
    else:
        proc = subprocess.run(['rio', 'merge', *glob.glob(infiles), outfile, '--co', 'compress=LZW',
                              '--co', 'tiled=true', '--co', 'blockxsize=256', '--co', 'blockysize=256'],
                              stderr=subprocess.PIPE, stdout=subprocess.PIPE)
    return

Merge the cleaned tiles of building footprints together into a single shapefile. We'll use `geopandas` to concatenate all the polygons together into a single geodataframe and then write out to a new shapefile.

In [ ]:
@dask.delayed
def merge_bldgs(*args, **kwargs):
    
    if os.path.exists(os.path.join(processed,'vectors','buildings.shp')):
        pass
    else:
        building_tiles = glob.glob(os.path.join(processed, 'vectors', 'building_tiles', '*.shp'))
        # create a list of geodataframes containing the tiles of building footprints
        gdflist = [gpd.read_file(tile) for tile in building_tiles]
        # merge them all together
        merged = gpd.GeoDataFrame(pd.concat(gdflist, ignore_index=True))
        # using pandas' concat caused us to lose projection information, so let's add that back in
        merged.crs = gdflist[0].crs
        # and write the merged data to a new shapefile
        merged.to_file(os.path.join(processed,'vectors','buildings.shp'))

    return

A single state that will depend upon the completion of the merged rasters and vectors.

In [ ]:
@dask.delayed
def merge_done(*args, **kwargs):
    return

In [ ]:
# building the computation receipe
merge_dsk = {}
merge_dsk['tiles_over'] = (tiles_overview, ['tiles_done'])
merge_dsk['merge_bldgs'] = (merge_bldgs, ['tiles_done'])
merge_dsk['merge_hill'] = (merge_hillshade, ['tiles_done'])
merge_dsk['merge_dem'] = (merge_dem, ['tiles_done'])
merge_dsk['merge_chm'] = (merge_chm, ['tiles_done'])
merge_dsk['merge_done']=(merge_done, ['tiles_over', 'merge_bldgs', 'merge_hill', 'merge_dem', 'merge_chm'])

In [ ]:
merge_graph = c.get(merge_dsk, 'merge_done') # build the computation graph
merge_results = c.compute(merge_graph) # this might take a while...
progress(merge_results)

In [ ]:
# c.cancel(res)

In [ ]:
c.close()
cluster.close()